In [1]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 8.4 MB/s eta 0:00:00


In [2]:
%matplotlib inline

import pandas as pd
import numpy as np
import math, time, random, datetime

import matplotlib.pyplot as plt

from sklearn.preprocessing import OneHotEncoder, LabelEncoder, label_binarize

import catboost
from sklearn import model_selection, tree, preprocessing, metrics, linear_model
from sklearn.model_selection import train_test_split


from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import AdaBoostRegressor


from catboost import CatBoostClassifier, Pool, cv

## Load data

In [3]:
data = pd.read_csv("/content/bike_machine_learning.csv")
data.head()

,tanggal,bulan,musim,tahun,holiday,status_hari_kerja,hari,situasi_cuaca,suhu,a_suhu,kelembapan,kecepatan_angin,tipe_pengendara,pengendara
0,2021-01-01,1,Dingin,2021,Bukan hari raya,Hari Libur,Sabtu,Cerah,0.24,0.2879,0.81,0.0,casual,3
1,2021-01-01,1,Dingin,2021,Bukan hari raya,Hari Libur,Sabtu,Cerah,0.22,0.2727,0.80,0.0,casual,8
2,2021-01-01,1,Dingin,2021,Bukan hari raya,Hari Libur,Sabtu,Cerah,0.22,0.2727,0.80,0.0,casual,5
3,2021-01-01,1,Dingin,2021,Bukan hari raya,Hari Libur,Sabtu,Cerah,0.24,0.2879,0.75,0.0,casual,3
4,2021-01-01,1,Dingin,2021,Bukan hari raya,Hari Libur,Sabtu,Cerah,0.24,0.2879,0.75,0.0,casual,0


In [4]:
data[:10000].tail()

,tanggal,bulan,musim,tahun,holiday,status_hari_kerja,hari,situasi_cuaca,suhu,a_suhu,kelembapan,kecepatan_angin,tipe_pengendara,pengendara
9995,2021-03-01,3,Dingin,2021,Bukan hari raya,Hari Kerja,Selasa,Cerah,0.32,0.3485,0.39,0.0,registered,65
9996,2021-03-01,3,Dingin,2021,Bukan hari raya,Hari Kerja,Selasa,Cerah,0.32,0.3333,0.36,0.0,registered,61
9997,2021-03-01,3,Dingin,2021,Bukan hari raya,Hari Kerja,Selasa,Cerah,0.34,0.3636,0.34,0.0,registered,57
9998,2021-03-01,3,Dingin,2021,Bukan hari raya,Hari Kerja,Selasa,Cerah,0.34,0.3485,0.34,0.0,registered,92
9999,2021-03-01,3,Dingin,2021,Bukan hari raya,Hari Kerja,Selasa,Cerah,0.34,0.3485,0.34,0.0,registered,230


## Pre-processing

In [5]:
len(data)

34758

In [6]:
34758/2

17379.0

In [7]:
first_year = data[:17290]

In [8]:
first_year

,tanggal,bulan,musim,tahun,holiday,status_hari_kerja,hari,situasi_cuaca,suhu,a_suhu,kelembapan,kecepatan_angin,tipe_pengendara,pengendara
0,2021-01-01,1,Dingin,2021,Bukan hari raya,Hari Libur,Sabtu,Cerah,0.24,0.2879,0.81,0.0,casual,3
1,2021-01-01,1,Dingin,2021,Bukan hari raya,Hari Libur,Sabtu,Cerah,0.22,0.2727,0.80,0.0,casual,8
2,2021-01-01,1,Dingin,2021,Bukan hari raya,Hari Libur,Sabtu,Cerah,0.22,0.2727,0.80,0.0,casual,5
3,2021-01-01,1,Dingin,2021,Bukan hari raya,Hari Libur,Sabtu,Cerah,0.24,0.2879,0.75,0.0,casual,3
4,2021-01-01,1,Dingin,2021,Bukan hari raya,Hari Libur,Sabtu,Cerah,0.24,0.2879,0.75,0.0,casual,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17285,2021-12-31,12,Dingin,2021,Bukan hari raya,Hari Libur,Sabtu,Cerah,0.42,0.4242,0.54,0.0,registered,73
17286,2021-12-31,12,Dingin,2021,Bukan hari raya,Hari Libur,Sabtu,Cerah,0.42,0.4242,0.54,0.0,registered,63
17287,2021-12-31,12,Dingin,2021,Bukan hari raya,Hari Libur,Sabtu,Cerah,0.40,0.4091,0.58,0.0,registered,50
17288,2021-12-31,12,Dingin,2021,Bukan hari raya,Hari Libur,Sabtu,Cerah,0.38,0.3939,0.62,0.0,registered,36


In [9]:
second_year = data[17290:]

In [10]:
second_year

,tanggal,bulan,musim,tahun,holiday,status_hari_kerja,hari,situasi_cuaca,suhu,a_suhu,kelembapan,kecepatan_angin,tipe_pengendara,pengendara
17290,2022-01-01,1,Dingin,2022,Bukan hari raya,Hari Libur,Minggu,Cerah,0.36,0.3788,0.66,0.0000,casual,5
17291,2022-01-01,1,Dingin,2022,Bukan hari raya,Hari Libur,Minggu,Cerah,0.36,0.3485,0.66,0.1343,casual,15
17292,2022-01-01,1,Dingin,2022,Bukan hari raya,Hari Libur,Minggu,Cerah,0.32,0.3485,0.76,0.0000,casual,16
17293,2022-01-01,1,Dingin,2022,Bukan hari raya,Hari Libur,Minggu,Cerah,0.30,0.3333,0.81,0.0000,casual,11
17294,2022-01-01,1,Dingin,2022,Bukan hari raya,Hari Libur,Minggu,Cerah,0.28,0.3030,0.81,0.0896,casual,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34753,2022-12-31,12,Dingin,2022,Bukan hari raya,Hari Kerja,Senin,Kabut/Awan,0.26,0.2576,0.60,0.1642,registered,108
34754,2022-12-31,12,Dingin,2022,Bukan hari raya,Hari Kerja,Senin,Kabut/Awan,0.26,0.2576,0.60,0.1642,registered,81
34755,2022-12-31,12,Dingin,2022,Bukan hari raya,Hari Kerja,Senin,Cerah,0.26,0.2576,0.60,0.1642,registered,83
34756,2022-12-31,12,Dingin,2022,Bukan hari raya,Hari Kerja,Senin,Cerah,0.26,0.2727,0.56,0.1343,registered,48


### Data preparation

In [11]:
first_year.columns

Index(['tanggal', 'bulan', 'musim', 'tahun', 'holiday', 'status_hari_kerja',
       'hari', 'situasi_cuaca', 'suhu', 'a_suhu', 'kelembapan',
       'kecepatan_angin', 'tipe_pengendara', 'pengendara'],
      dtype='object')

In [12]:
data_new_drop = first_year.drop(['tanggal', 'tahun', 'bulan', 'hari'], axis=1)

In [13]:
data_new_drop.head()

,musim,holiday,status_hari_kerja,situasi_cuaca,suhu,a_suhu,kelembapan,kecepatan_angin,tipe_pengendara,pengendara
0,Dingin,Bukan hari raya,Hari Libur,Cerah,0.24,0.2879,0.81,0.0,casual,3
1,Dingin,Bukan hari raya,Hari Libur,Cerah,0.22,0.2727,0.80,0.0,casual,8
2,Dingin,Bukan hari raya,Hari Libur,Cerah,0.22,0.2727,0.80,0.0,casual,5
3,Dingin,Bukan hari raya,Hari Libur,Cerah,0.24,0.2879,0.75,0.0,casual,3
4,Dingin,Bukan hari raya,Hari Libur,Cerah,0.24,0.2879,0.75,0.0,casual,0


In [14]:
data_new_drop_2 = second_year.drop(['tanggal', 'tahun', 'bulan', 'hari'], axis=1)

In [15]:
data_new_drop_2.head()

,musim,holiday,status_hari_kerja,situasi_cuaca,suhu,a_suhu,kelembapan,kecepatan_angin,tipe_pengendara,pengendara
17290,Dingin,Bukan hari raya,Hari Libur,Cerah,0.36,0.3788,0.66,0.0000,casual,5
17291,Dingin,Bukan hari raya,Hari Libur,Cerah,0.36,0.3485,0.66,0.1343,casual,15
17292,Dingin,Bukan hari raya,Hari Libur,Cerah,0.32,0.3485,0.76,0.0000,casual,16
17293,Dingin,Bukan hari raya,Hari Libur,Cerah,0.30,0.3333,0.81,0.0000,casual,11
17294,Dingin,Bukan hari raya,Hari Libur,Cerah,0.28,0.3030,0.81,0.0896,casual,0


## One-hot Encoding


Ubah ke dalam format one-hot ==> musim, holiday, status_hari_kerja, situasi_cuaca, dan	tipe_pengendara

In [16]:
musim_one_hot = pd.get_dummies(data_new_drop['musim'])
holiday_one_hot = pd.get_dummies(data_new_drop['holiday'])
status_one_hot = pd.get_dummies(data_new_drop['status_hari_kerja'])
cuaca_one_hot = pd.get_dummies(data_new_drop['situasi_cuaca'])
tipe_one_hot = pd.get_dummies(data_new_drop['tipe_pengendara'])

In [17]:
musim_one_hot_2 = pd.get_dummies(data_new_drop_2['musim'])
holiday_one_hot_2 = pd.get_dummies(data_new_drop_2['holiday'])
status_one_hot_2 = pd.get_dummies(data_new_drop_2['status_hari_kerja'])
cuaca_one_hot_2 = pd.get_dummies(data_new_drop_2['situasi_cuaca'])
tipe_one_hot_2 = pd.get_dummies(data_new_drop_2['tipe_pengendara'])

In [18]:
musim_one_hot.head()

,Dingin,Gugur,Panas,Semi
0,True,False,False,False
1,True,False,False,False
2,True,False,False,False
3,True,False,False,False
4,True,False,False,False


In [19]:
holiday_one_hot.head()

,Bukan hari raya,Hari raya
0,True,False
1,True,False
2,True,False
3,True,False
4,True,False


In [20]:
status_one_hot.head()

,Hari Kerja,Hari Libur
0,False,True
1,False,True
2,False,True
3,False,True
4,False,True


In [21]:
cuaca_one_hot.head()

,Cerah,Hujan/Salju Lebat,Hujan/Salju Ringan,Kabut/Awan
0,True,False,False,False
1,True,False,False,False
2,True,False,False,False
3,True,False,False,False
4,True,False,False,False


In [22]:
tipe_one_hot.head()

,casual,registered
0,True,False
1,True,False
2,True,False
3,True,False
4,True,False


In [23]:
tipe_one_hot_2.head()

,casual,registered
17290,True,False
17291,True,False
17292,True,False
17293,True,False
17294,True,False


Hapus kolom yang sudah diubah dan masukkan kolom transformasi one-hot

In [24]:
data_new_drop = data_new_drop.drop(['musim', 'holiday', 'status_hari_kerja', 'situasi_cuaca', 'tipe_pengendara'], axis=1)
data_new_drop.head()

,suhu,a_suhu,kelembapan,kecepatan_angin,pengendara
0,0.24,0.2879,0.81,0.0,3
1,0.22,0.2727,0.80,0.0,8
2,0.22,0.2727,0.80,0.0,5
3,0.24,0.2879,0.75,0.0,3
4,0.24,0.2879,0.75,0.0,0


In [25]:
data_new_drop_2 = data_new_drop_2.drop(['musim', 'holiday', 'status_hari_kerja', 'situasi_cuaca', 'tipe_pengendara'], axis=1)
data_new_drop_2.head()

,suhu,a_suhu,kelembapan,kecepatan_angin,pengendara
17290,0.36,0.3788,0.66,0.0000,5
17291,0.36,0.3485,0.66,0.1343,15
17292,0.32,0.3485,0.76,0.0000,16
17293,0.30,0.3333,0.81,0.0000,11
17294,0.28,0.3030,0.81,0.0896,0


In [26]:
data_new = pd.concat([data_new_drop, musim_one_hot, holiday_one_hot, status_one_hot, cuaca_one_hot, tipe_one_hot], axis=1)
data_new.head()

,suhu,a_suhu,kelembapan,kecepatan_angin,pengendara,Dingin,Gugur,Panas,Semi,Bukan hari raya,Hari raya,Hari Kerja,Hari Libur,Cerah,Hujan/Salju Lebat,Hujan/Salju Ringan,Kabut/Awan,casual,registered
0,0.24,0.2879,0.81,0.0,3,True,False,False,False,True,False,False,True,True,False,False,False,True,False
1,0.22,0.2727,0.80,0.0,8,True,False,False,False,True,False,False,True,True,False,False,False,True,False
2,0.22,0.2727,0.80,0.0,5,True,False,False,False,True,False,False,True,True,False,False,False,True,False
3,0.24,0.2879,0.75,0.0,3,True,False,False,False,True,False,False,True,True,False,False,False,True,False
4,0.24,0.2879,0.75,0.0,0,True,False,False,False,True,False,False,True,True,False,False,False,True,False


In [27]:
data_new_2 = pd.concat([data_new_drop_2, musim_one_hot_2, holiday_one_hot_2, status_one_hot_2, cuaca_one_hot_2, tipe_one_hot_2], axis=1)
data_new_2.head()

,suhu,a_suhu,kelembapan,kecepatan_angin,pengendara,Dingin,Gugur,Panas,Semi,Bukan hari raya,Hari raya,Hari Kerja,Hari Libur,Cerah,Hujan/Salju Lebat,Hujan/Salju Ringan,Kabut/Awan,casual,registered
17290,0.36,0.3788,0.66,0.0000,5,True,False,False,False,True,False,False,True,True,False,False,False,True,False
17291,0.36,0.3485,0.66,0.1343,15,True,False,False,False,True,False,False,True,True,False,False,False,True,False
17292,0.32,0.3485,0.76,0.0000,16,True,False,False,False,True,False,False,True,True,False,False,False,True,False
17293,0.30,0.3333,0.81,0.0000,11,True,False,False,False,True,False,False,True,True,False,False,False,True,False
17294,0.28,0.3030,0.81,0.0896,0,True,False,False,False,True,False,False,True,True,False,False,False,True,False


In [28]:
second_year[:4358].tail()

,tanggal,bulan,musim,tahun,holiday,status_hari_kerja,hari,situasi_cuaca,suhu,a_suhu,kelembapan,kecepatan_angin,tipe_pengendara,pengendara
21643,2022-06-30,6,Panas,2022,Bukan hari raya,Hari Libur,Sabtu,Cerah,0.84,0.7576,0.44,0.2537,casual,134
21644,2022-06-30,6,Panas,2022,Bukan hari raya,Hari Libur,Sabtu,Cerah,0.82,0.7727,0.52,0.1642,casual,88
21645,2022-06-30,6,Panas,2022,Bukan hari raya,Hari Libur,Sabtu,Cerah,0.82,0.7727,0.52,0.1642,casual,48
21646,2022-06-30,6,Panas,2022,Bukan hari raya,Hari Libur,Sabtu,Cerah,0.78,0.7424,0.62,0.1642,casual,38
21647,2022-06-30,6,Panas,2022,Bukan hari raya,Hari Libur,Sabtu,Cerah,0.78,0.7424,0.62,0.0896,casual,33


In [29]:
# Buat data latih dan target
X_train = data_new.drop(['pengendara'], axis=1)
y_train = data_new['pengendara']

# prediksi menggunakan data 10000 data awal pada tahun pertama
prediction_data = data_new[:10000]
X_test = prediction_data.drop(['pengendara'], axis=1)
y_test = prediction_data['pengendara']

# prediksi menggunakan data 6 bulan awal dari tahun kedua
prediction_data_2 = data_new_2[:4358]
X_test_2 = prediction_data_2.drop(['pengendara'], axis=1)
y_test_2 = prediction_data_2['pengendara']

## Model Training

In [66]:
import time
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

def fit_ml(algo, X_train, y_train, cv):
    start_time = time.time()
    model = algo.fit(X_train, y_train)
    acc = round(model.score(X_train, y_train) * 100, 2)

    # cross validation
    train_pred = model_selection.cross_val_predict(algo,
                                                   X_train,
                                                   y_train, cv=cv,
                                                   n_jobs=-1)
    # Cross validation accuracy matrix
    mse = round(metrics.mean_squared_error(y_train, train_pred) * 100, 2)
    mae = round(metrics.mean_absolute_error(y_train, train_pred) * 100, 2)

    # prediksi tahun 1
    y_pred = model.predict(X_test)
    comparison = pd.concat([y_test, pd.Series(y_pred, name='prediction')], axis=1)
    comparison.to_csv(str(algo) + '.csv', index=False)

    # prediksi 6 bulan tahun kedua
    y_pred_2 = model.predict(X_test_2)
    reset = y_test.reset_index(drop=True)
    comparison_2 = pd.concat([reset, pd.Series(y_pred_2, name='prediction')], axis=1)
    comparison_2.to_csv(str(algo) + '_2.csv', index=False)


    algo_time = (time.time() - start_time)

    print(f"Accuracy: {acc}")
    print(f"MSE: {mse}")
    print(f"Running time: {datetime.timedelta(seconds=algo_time)}")

    return train_pred, acc, mse, algo_time

#### Linear Regression

In [67]:
train_pred_linear, acc_linear, acc_cv_linear, algo_time_linear = fit_ml(LinearRegression(), X_train, y_train, 10)

Accuracy: 37.96
MSE: 580394.37
Running time: 0:00:00.455161


#### MLP Regressor

In [68]:
train_pred_mlp, acc_mlp, acc_cv_mlp, algo_time_mlp = fit_ml(MLPRegressor(), X_train, y_train, 10)

/usr/local/lib/python3.12/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Accuracy: 45.59
MSE: 514398.11
Running time: 0:02:09.063576


#### Decision Tree Regressor

In [69]:
train_pred_dt, acc_dt, acc_cv_dt, algo_time_dt = fit_ml(DecisionTreeRegressor(), X_train, y_train, 10)

Accuracy: 72.53
MSE: 939076.71
Running time: 0:00:00.743514


#### K-Neighbors Regressor

In [70]:
train_pred_knn, acc_knn, acc_cv_knn, algo_time_knn = fit_ml(KNeighborsRegressor(), X_train, y_train, 10)

Accuracy: 59.05
MSE: 646822.55
Running time: 0:00:05.501580


#### Support Vector Regressor

In [71]:
train_pred_svr, acc_svr, acc_cv_svr, algo_time_svr = fit_ml(SVR(), X_train, y_train, 10)

Accuracy: 36.01
MSE: 571186.6
Running time: 0:03:49.425497


#### Random Forest Regressor

In [72]:
train_pred_rf, acc_rf, acc_cv_rf, algo_time_rf = fit_ml(RandomForestRegressor(), X_train, y_train, 10)

Accuracy: 70.72
MSE: 674611.56
Running time: 0:00:42.775788


#### Gradient Boosting Regressor

In [73]:
train_pred_gb, acc_gb, acc_cv_gb, algo_time_gb = fit_ml(GradientBoostingRegressor(), X_train, y_train, 10)

Accuracy: 47.55
MSE: 510945.29
Running time: 0:00:13.595749


#### Hist Gradient Boosting Regressor

In [74]:
train_pred_hgb, acc_hgb, acc_cv_hgb, algo_time_hgb = fit_ml(HistGradientBoostingRegressor(), X_train, y_train, 10)

Accuracy: 52.41
MSE: 546662.04
Running time: 0:00:03.992920


#### Extra Tree Regressor

In [75]:
train_pred_et, acc_et, acc_cv_et, algo_time_et = fit_ml(ExtraTreesRegressor(), X_train, y_train, 10)

Accuracy: 72.53
MSE: 743693.97
Running time: 0:00:37.530606


#### Bagging Regressor

In [76]:
train_pred_br, acc_br, acc_cv_br, algo_time_br = fit_ml(BaggingRegressor(), X_train, y_train, 10)

Accuracy: 69.35
MSE: 702034.54
Running time: 0:00:04.678400


#### AdBoost Regressor

In [77]:
train_pred_ab, acc_ab, acc_cv_ab, algo_time_ab = fit_ml(AdaBoostRegressor(), X_train, y_train, 10)

Accuracy: 36.5
MSE: 653939.1
Running time: 0:00:01.937237


In [78]:
# CatBoost Regression Model
from catboost import CatBoostRegressor

# Initialize the CatBoostRegressor with RMSE as the loss function
model = CatBoostRegressor(loss_function='RMSE')

# Fit the model on the training data with verbose logging every 100 iterations
model.fit(X_train, y_train, verbose=100)

Learning rate set to 0.064232
0:	learn: 90.5469714	total: 51.2ms	remaining: 51.2s
100:	learn: 66.6275175	total: 370ms	remaining: 3.29s
200:	learn: 65.5405532	total: 685ms	remaining: 2.72s
300:	learn: 64.7260194	total: 1.01s	remaining: 2.35s
400:	learn: 64.1162614	total: 1.31s	remaining: 1.96s
500:	learn: 63.6614658	total: 1.6s	remaining: 1.59s
600:	learn: 63.2732159	total: 1.92s	remaining: 1.27s
700:	learn: 62.9067324	total: 2.21s	remaining: 945ms
800:	learn: 62.5883014	total: 2.51s	remaining: 623ms
900:	learn: 62.2806977	total: 2.83s	remaining: 311ms
999:	learn: 61.9950864	total: 3.11s	remaining: 0us


## Prediksi

In [79]:
def predict_data(path):
  model_1 = pd.read_csv(path)
  model_1.head()
  pred = []
  for i in range(len(model_1)):
    prediction = math.isclose(model_1['pengendara'][i],model_1['prediction'][i], abs_tol=1.0)
    pred.append(prediction)
    #print(model_1['pengendara'][i], model_1['prediction'][i], prediction)
  right = 0
  for i in pred:
    if i == True:
      right += 1
    else:
      pass
  wrong = len(pred)-right
  #print(f"benar: {right}")
  #print(f"salah: {wrong}")

In [80]:
path = "/content/RandomForestRegressor().csv"
model_1 = pd.read_csv(path)
model_1.head()

,pengendara,prediction
0,5.0,NaN
1,15.0,NaN
2,16.0,NaN
3,11.0,NaN
4,0.0,NaN


In [81]:
model_1.tail(10)

,pengendara,prediction
14348,NaN,115.328833
14349,NaN,113.164083
14350,NaN,91.092833
14351,NaN,74.683333
14352,NaN,92.879048
14353,NaN,69.324167
14354,NaN,78.868500
14355,NaN,88.339000
14356,NaN,138.921667
14357,NaN,138.921667


In [82]:
#import pandas as pd
#data = pd.read_csv("/content/bike_machine_learning.csv")

In [83]:
data.tail(10)

,tanggal,bulan,musim,tahun,holiday,status_hari_kerja,hari,situasi_cuaca,suhu,a_suhu,kelembapan,kecepatan_angin,tipe_pengendara,pengendara
34748,2022-12-31,12,Dingin,2022,Bukan hari raya,Hari Kerja,Senin,Kabut/Awan,0.28,0.2727,0.45,0.2239,registered,185
34749,2022-12-31,12,Dingin,2022,Bukan hari raya,Hari Kerja,Senin,Kabut/Awan,0.28,0.2879,0.45,0.1343,registered,246
34750,2022-12-31,12,Dingin,2022,Bukan hari raya,Hari Kerja,Senin,Kabut/Awan,0.26,0.2576,0.48,0.1940,registered,184
34751,2022-12-31,12,Dingin,2022,Bukan hari raya,Hari Kerja,Senin,Kabut/Awan,0.26,0.2879,0.48,0.0896,registered,150
34752,2022-12-31,12,Dingin,2022,Bukan hari raya,Hari Kerja,Senin,Kabut/Awan,0.26,0.2727,0.48,0.1343,registered,112
34753,2022-12-31,12,Dingin,2022,Bukan hari raya,Hari Kerja,Senin,Kabut/Awan,0.26,0.2576,0.60,0.1642,registered,108
34754,2022-12-31,12,Dingin,2022,Bukan hari raya,Hari Kerja,Senin,Kabut/Awan,0.26,0.2576,0.60,0.1642,registered,81
34755,2022-12-31,12,Dingin,2022,Bukan hari raya,Hari Kerja,Senin,Cerah,0.26,0.2576,0.60,0.1642,registered,83
34756,2022-12-31,12,Dingin,2022,Bukan hari raya,Hari Kerja,Senin,Cerah,0.26,0.2727,0.56,0.1343,registered,48
34757,2022-12-31,12,Dingin,2022,Bukan hari raya,Hari Kerja,Senin,Cerah,0.26,0.2727,0.65,0.1343,registered,37
